In [1]:
#Instalação
!pipenv install yfinance==0.2.41
!pipenv install crewai==0.28.8
!pipenv install 'crewai[tools]'
!pipenv install langchain==0.1.20
!pipenv install langchain-openai==0.1.7
!pipenv install langchain-community==0.0.38
!pipenv install duckduckgo-search==5.3.0

Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Creating a Pipfile for this project...
Installing yfinance==0.2.41...
Resolving yfinance==0.2.41...
Added yfinance to Pipfile's [packages] ...
✔ Installation Succeededce...
⠋ Installing yfinance...
Pipfile.lock not found, creating...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠋ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (195e77640cf3cc8228d1598b6efd00e40334ab5ffe624844dac8e22f96e545d8)!
Installing dependencies from Pipfile.lock (e545d8)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Co

In [2]:
import json
import os
from datetime import datetime

import yfinance as yf
from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

/Users/brunoramoslemos/.local/share/virtualenvs/ai-agent-JTRlUzcV/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [3]:
def fetch_stock_price(ticket):
  stocks = yf.download(ticket, start="2023-08-08", end="2024-08-08")
  return stocks

yahoo_finance_tool = Tool(
  name = "Yahoo Finance Tool",
  description = "Fetches stock prices for {ticket} from the last year about a specific stock from Yahoo Finance API",
  func = lambda ticket: fetch_stock_price(ticket)
)

In [4]:
response = yahoo_finance_tool.run("AAPL")

[*********************100%%**********************]  1 of 1 completed


In [5]:
print(response)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643402   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043747   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546356   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date         

In [36]:
os.environ['OPENAI_API_KEY'] = ""


llm = ChatOpenAI(model="gpt-3.5-turbo")

In [37]:
stockPriceAnalyst = Agent(
  role = "Senior stock price Analyst",
  goal = "Find the {ticket} stock price and analyses trends",
  backstory= """You're a highly experienced in analyzing the price of a specific stock an make
  predictions about its future price
  """,
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  tools = [yahoo_finance_tool],
  allow_delegation=False
)

In [38]:
getStockPrice = Task(
  description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
  expected_output = """Specify the current trend stock price - up, down or sideways.
  eg. stock = 'APPL, price UP'
  """,
  agent= stockPriceAnalyst
)

In [39]:
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [40]:
newsAnalyst = Agent(
  role = "Stock News Analyst",
  goal = """Create a short summary of the market news related to the stock {ticket} company. 
  Specify the current trend - up, down or sideways with the news context. For each request stock specify a number 
  between 0 and 100, where 0 is extreme fear and 100 is extreme greed.
  """,
  backstory= """You're a highly experienced in analyzing the market trends and news and have tracked assets for more than 10 years.
  
  You're also master level analyst in the tradicional markets and have deep understanding of humam psychology. 
  
  You understand news, their titles and information, but you look at those with a health and skepticism.
  You condider also the source of the news articles.
  """,
  verbose = True,
  llm = llm,
  max_iter = 10,
  memory = True,
  tools = [search_tool],
  allow_delegation=False
)

In [41]:
get_news = Task(
  description=f"""Take the stock and always include BTC to it (if not requested)
  Use the search tool to search each one individually.
  
  The current date is {datetime.now()}.
  
  Compose the results into a helpfull report.
  """,
  expected_output = """A summary of the overall market and one sentence summary for each request asset.
  Include a fear/greed score for each asset based on the news. Use format:
  <STOCK ASSET>
  <SUMMARY BASED ON NEWS>
  <TREND PREDICTION>
  <FEAR/GREED SCORE>
  """,
  agent= newsAnalyst
)

In [42]:
stockAnalystWrite = Agent(
  role = "Senior Stock Analyst Writer",
  goal = """ 
  Analyze the trends price and news and write an insighfull, compelling and informative 3 paragraph long  newsletter based on the stock report and price trend.
  """,
  backstory = """You're widely accepted as the best stock analyst in the market. You understamd complex  concepts and create compelling stories and narratives tha resonate with wider audiences.
  
  You understand macro factors and combina multiple theories - eg. cycle theory and fundamental analyses.
  You're able to hold multiple opinions when analyzing anything
  """,
  verbose=True,
  llm=llm,
  max_iter=5,
  memory=True,
  allow_delegation=True
)

In [43]:
writeAnalyses = Task(
  description="""
  Use the stock price trend an the stock news report to create a analyses and write the newsletter about the {ticket} company
  that is brief and highlights the most important points.
  Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
  Include the previous analyses of stock trend and news summary.
  """,
  expected_output="""
    An eloquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contain:
    
    - 3 bullets executive summary
    - Introduction - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fear/greed scores
    - summary = key facts and concrete future trend predictions - up, down or sideways
  """,
  agent = stockAnalystWrite,
  context = [getStockPrice, get_news]
)

In [44]:
crew = Crew(
  agents=[stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
  tasks=[getStockPrice, get_news, writeAnalyses],
  verbose = True,
  process= Process.hierarchical,
  full_output=True,
  share_crew=False,
  manager_llm=llm,
  max_iter=15
)

2024-08-19 21:34:13,831 - 8165764160 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [45]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})

 [2024-08-19 21:34:15][DEBUG]: Working Agent: Crew Manager
 [2024-08-19 21:34:15][INFO]: Starting Task: Analyze the stock AAPL price history and create a trend analyses of up, down or sideways


> Entering new CrewAgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [46]:
results

CrewOutput(raw='Agent stopped due to iteration limit or time limit.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Analyze the stock AAPL price history and create a trend analyses of up, down or sideways', summary='Analyze the stock AAPL price history and create a trend...', raw='Agent stopped due to iteration limit or time limit.', pydantic=None, json_dict=None, agent='Crew Manager', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='Take the stock and always include BTC to it (if not requested)\n  Use the search tool to search each one individually.\n  \n  The current date is 2024-08-19 21:00:53.743060.\n  \n  Compose the results into a helpfull report.\n  ', summary='Take the stock and always include BTC to it (if...', raw='Agent stopped due to iteration limit or time limit.', pydantic=None, json_dict=None, agent='Crew Manager', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='\n  Use the stock price trend an the stock news rep